In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import numpy as np

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [9]:
import pandas as pd

# --- 1. Cargar y Seleccionar ---
file_path = r'D:\OneDrive\data-projects\analisis-powerlifters-chilenos\data\powerliftingchile.csv'
df_raw = pd.read_csv(file_path)

# Seleccionamos las columnas de características y nuestros TRES nuevos objetivos
cols_necesarias = [
    'Age', 'BodyweightKg', 'Sex', 'Equipment',  # Características
    'Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg' # Objetivos
]
df_limpieza = df_raw[cols_necesarias].copy()

# --- 2. Limpiar Datos ---
print(f"Registros antes de la limpieza: {len(df_limpieza)}")

# Eliminamos cualquier fila que tenga un valor nulo en CUALQUIERA de estas columnas
df_limpio = df_limpieza.dropna()
print(f"Registros después de eliminar nulos: {len(df_limpio)}")

# Eliminamos filas donde cualquier levantamiento sea un intento fallido (negativo o cero)
df_limpio = df_limpio[
    (df_limpio['Best3SquatKg'] > 0) &
    (df_limpio['Best3BenchKg'] > 0) &
    (df_limpio['Best3DeadliftKg'] > 0)
]
print(f"Registros después de eliminar levantamientos inválidos: {len(df_limpio)}")

# --- 3. Revisar el Resultado ---
print("\nDataFrame limpio y listo para el modelo multiobjetivo:")
display(df_limpio.head())

Registros antes de la limpieza: 1377
Registros después de eliminar nulos: 849
Registros después de eliminar levantamientos inválidos: 847

DataFrame limpio y listo para el modelo multiobjetivo:


,Age,BodyweightKg,Sex,Equipment,Best3SquatKg,Best3BenchKg,Best3DeadliftKg
171,17.5,72.58,M,Raw,205.0,112.5,250.0
172,32.0,67.20,F,Raw,132.5,92.5,152.5
173,25.5,73.30,M,Raw,252.5,175.0,312.5
174,29.5,65.74,M,Raw,220.0,140.0,250.0
175,30.5,82.16,M,Raw,290.0,195.0,300.0


In [11]:
# X son las mismas características predictoras
X = df_limpio[['Age', 'BodyweightKg', 'Sex', 'Equipment']]

# y ahora son nuestros tres objetivos
y = df_limpio[['Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg']]

print("Características (X):")
display(X.head())
print("\nObjetivos (y):")
display(y.head())

Características (X):


,Age,BodyweightKg,Sex,Equipment
171,17.5,72.58,M,Raw
172,32.0,67.20,F,Raw
173,25.5,73.30,M,Raw
174,29.5,65.74,M,Raw
175,30.5,82.16,M,Raw



Objetivos (y):


,Best3SquatKg,Best3BenchKg,Best3DeadliftKg
171,205.0,112.5,250.0
172,132.5,92.5,152.5
173,252.5,175.0,312.5
174,220.0,140.0,250.0
175,290.0,195.0,300.0


In [12]:
# Identificar las características categóricas que necesitan ser transformadas
categorical_features = ['Sex', 'Equipment']

# Crear el transformador para las variables categóricas (OneHotEncoder)
# OneHotEncoder crea nuevas columnas para cada categoría (ej. Sex_M, Sex_F)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Mantiene las columnas numéricas (Age, BodyweightKg) sin cambios
)

# Definir el modelo que vamos a usar
# RandomForestRegressor es una excelente opción para este tipo de problema.
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Ensamblar los pasos en un único pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', model)])

print("Pipeline de preprocesamiento y modelo creado:")
print(pipeline)

Pipeline de preprocesamiento y modelo creado:
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'Equipment'])])),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])


In [13]:
# Dividir los datos: 80% para entrenar, 20% para probar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Set de entrenamiento: {len(X_train)} registros.")
print(f"Set de prueba: {len(X_test)} registros.")

# Entrenar el pipeline con los datos de entrenamiento
print("\nEntrenando el modelo... (esto podría tomar unos segundos)")
pipeline.fit(X_train, y_train)

print("¡Entrenamiento completado!")

Set de entrenamiento: 677 registros.
Set de prueba: 170 registros.

Entrenando el modelo... (esto podría tomar unos segundos)
¡Entrenamiento completado!


In [16]:
# Dividir los datos (si no lo has hecho al re-ejecutar la celda 5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo (si no lo has hecho)
# pipeline.fit(X_train, y_train)

# Realizar predicciones. 'y_pred' será un array con 3 columnas.
y_pred = pipeline.predict(X_test)

# Calcular R² general
r2 = r2_score(y_test, y_pred)

# Calcular el Error Absoluto Medio (MAE) para cada levantamiento
mae_squat = mean_absolute_error(y_test['Best3SquatKg'], y_pred[:, 0])
mae_bench = mean_absolute_error(y_test['Best3BenchKg'], y_pred[:, 1])
mae_deadlift = mean_absolute_error(y_test['Best3DeadliftKg'], y_pred[:, 2])

print("--- 📊 Evaluación del Modelo Multiobjetivo ---")
print(f"R² General (promedio de todos los objetivos): {r2:.2f}")
print("\n--- Error Promedio por Levantamiento (MAE) ---")
print(f"🏋️‍♂️ Sentadilla: El modelo se equivoca en promedio por {mae_squat:.2f} kg.")
print(f"🏋️‍♂️ Press de Banca: El modelo se equivoca en promedio por {mae_bench:.2f} kg.")
print(f"🏋️‍♂️ Peso Muerto: El modelo se equivoca en promedio por {mae_deadlift:.2f} kg.")

--- 📊 Evaluación del Modelo Multiobjetivo ---
R² General (promedio de todos los objetivos): 0.77

--- Error Promedio por Levantamiento (MAE) ---
🏋️‍♂️ Sentadilla: El modelo se equivoca en promedio por 24.20 kg.
🏋️‍♂️ Press de Banca: El modelo se equivoca en promedio por 14.36 kg.
🏋️‍♂️ Peso Muerto: El modelo se equivoca en promedio por 21.78 kg.


In [18]:
# Guardar el nuevo pipeline multiobjetivo
model_filename = 'powerlifting_multi_model.joblib'
joblib.dump(pipeline, model_filename)

print(f"✅ ¡Nuevo modelo multiobjetivo guardado como '{model_filename}'!")

✅ ¡Nuevo modelo multiobjetivo guardado como 'powerlifting_multi_model.joblib'!
